# Advent of code 2022

Technically this should probably be modelled with an array or something, but the grid's small enough to use a dictionary and might make things slightly easier:

In [1]:
from collections import defaultdict
import re

## Part 1

In [2]:
test_input='''498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9'''

In [3]:
with open('data/day14.txt') as fIn:
    puzzle_input=fIn.read()

In [4]:
def parse_input(str_in):
    return [[(int(x.split(',')[0]), int(x.split(',')[1])) for x in nl.split('->')]
            for nl in str_in.splitlines()]

parse_input(test_input)

[[(498, 4), (498, 6), (496, 6)], [(503, 4), (502, 4), (502, 9), (494, 9)]]

In [5]:
def build_cave(str_in):
    
    list_rep=parse_input(str_in)
    
    blocked=defaultdict(int)
    
    for cc in parse_input(str_in):
    
        for ((x1, y1), (x2, y2)) in zip(cc, cc[1:]):
            assert (x1==x2) or (y1==y2)
            if x1==x2:
                for y in range(min(y1, y2), max(y1, y2)):
                    blocked[(x1, y)]=1
            else:
                for x in range(min(x1, x2), max(x1, x2)):
                    blocked[(x, y1)]=1
            blocked[(x1, y1)]=1
            blocked[(x2, y2)]=1
        
    return blocked
            
build_cave(test_input)

defaultdict(int,
            {(498, 4): 1,
             (498, 5): 1,
             (498, 6): 1,
             (496, 6): 1,
             (497, 6): 1,
             (502, 4): 1,
             (503, 4): 1,
             (502, 5): 1,
             (502, 6): 1,
             (502, 7): 1,
             (502, 8): 1,
             (502, 9): 1,
             (494, 9): 1,
             (495, 9): 1,
             (496, 9): 1,
             (497, 9): 1,
             (498, 9): 1,
             (499, 9): 1,
             (500, 9): 1,
             (501, 9): 1})

In [6]:
def sand_grain(x, y, caves, limit=1000):
    '''path of sand from x, y'''
    if y==limit:
        return False
    elif caves[(x, y+1)]==0:
        return sand_grain(x, y+1, caves)
    elif caves[(x-1, y+1)]==0:
        return sand_grain(x-1, y+1, caves)
    elif caves[(x+1, y+1)]==0:
        return sand_grain(x+1, y+1, caves)
    else:
        caves[(x, y)]=2
        return True

c=build_cave(test_input)
sand_grain(500, 0, c)
c

defaultdict(int,
            {(498, 4): 1,
             (498, 5): 1,
             (498, 6): 1,
             (496, 6): 1,
             (497, 6): 1,
             (502, 4): 1,
             (503, 4): 1,
             (502, 5): 1,
             (502, 6): 1,
             (502, 7): 1,
             (502, 8): 1,
             (502, 9): 1,
             (494, 9): 1,
             (495, 9): 1,
             (496, 9): 1,
             (497, 9): 1,
             (498, 9): 1,
             (499, 9): 1,
             (500, 9): 1,
             (501, 9): 1,
             (500, 1): 0,
             (500, 2): 0,
             (500, 3): 0,
             (500, 4): 0,
             (500, 5): 0,
             (500, 6): 0,
             (500, 7): 0,
             (500, 8): 2})

In [7]:
c=build_cave(test_input)

for _ in range(5):
    sand_grain(500, 0, c)

[x for x in c if c[x]==2]

[(500, 7), (500, 8), (499, 8), (501, 8), (498, 8)]

Looks OK. Should really work out the limits of the map, but for the moment, I'll just hack it by putting in a limit of 1000.

In [8]:
def day14_a(str_in):
    
    caves=build_cave(str_in)
    
    i=0
    while sand_grain(500, 0, caves):
        i+=1
    
    return i
    

In [9]:
assert day14_a(test_input)==24

In [10]:
day14_a(puzzle_input)

692

## Part 2

OK, very much not surprised that I've got to find the limits of the cave :-). So, let's do that properly:

In [11]:
def cave_system_bounds(caves):
    xs=[x for (x, _) in caves.keys()]
    ys=[y for (_, y) in caves.keys()]
    return {'x_min':min(xs),
            'x_max':max(xs),
            'y_min':min(ys),
            'y_max':max(ys)}

cave_system_bounds(build_cave(test_input))
    

{'x_min': 494, 'x_max': 503, 'y_min': 4, 'y_max': 9}

Tweak the sand falling function so that grains stop when they hit the floor. Also, let's return the point at which the grain of sand comes to rest.

In [12]:
def sand_grain_2(x, y, caves, limit):
    '''path of sand from x, y'''

    # Update so that if the grain is on the floor, then stop

    if y==limit:
        caves[(x, y)]=2
        return (x, y)
    elif caves[(x, y+1)]==0:
        return sand_grain_2(x, y+1, caves, limit)
    elif caves[(x-1, y+1)]==0:
        return sand_grain_2(x-1, y+1, caves, limit)
    elif caves[(x+1, y+1)]==0:
        return sand_grain_2(x+1, y+1, caves, limit)
    else:
        caves[(x, y)]=2
        return (x, y)


In [13]:
def day14_b(str_in):
    
    caves=build_cave(str_in)
    
    limit=cave_system_bounds(caves)['y_max']+1
    
    i=0
    while sand_grain_2(500, 0, caves, limit) != (500, 0):
        i+=1
    
    return i+1 # Need the final grain as well
    

In [14]:
assert day14_b(test_input)==93

In [15]:
day14_b(puzzle_input)

31706